<a href="https://colab.research.google.com/github/bes82/Deformable_Nondeformable/blob/main/Deformable_Nondeformable_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries, download network, and load network.

This cell has three jobs. 

1.) Import useful libraries for use during the pipeline.

2.) Download the pipeline network architecture and weights from a Google Drive.

3.) Load the network for use in the pipeline.

In [ ]:
#@title  <- Click here to run code. Double click this text to see code. 

print("Importing Useful Libraries...")
# Importing Useful Libraries.
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.models import load_model, Model, model_from_json

import cv2, os
from scipy.ndimage import label
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import shutil
print("Done!")

# Clone Repository And Download Network Architecture and Weights.
print("Cloning Repository...")
! git clone https://github.com/bes82/Deformable_Nondeformable
print("Done!")
%cd Deformable_Nondeformable

ce_jaccard_Ids = ["1mtewaKMXqdIVmWj1mXKQfYK9Tr5OCD7J", "1lQHqgcRCDhmAL5BrxSyfYwtejmpt_rXP",
"1K1cduESQHDozHZYInEE0PmREdvvahiRs", "1rwJ9PKUuZAw9ZON9u6P7KC9H05jIT1nZ",
"1Tn0e3OPhmFjZXLksP2reSTls3CFwoXSv", "1RrhIiRtWYFGJTpq-IRTXYyldaO0fKXFN",
"1-R1kK7Z89XVSSpraTS2DfT8nFXqek8Ar", "127f8DRvLuJ33V5_i-TtOZ4-SKrPzsCsT",
"1EWPauaixDCOHcDdN8LyBYj_CrtgCn6MX", "1N2e2sNJ8agWAiEX9Fw4gjEwk5OcFo_RY"]

resnet50_IDs = ["1G4LfnQQzn0ufnJkWvpEGmbdqXCZ02YKn", "1P16p6KZt04chEt8griUQc-A6n7f4H6O1",
"1znKMg6KqkNGniIRtL6d3qEV41ZAJWrG4", "15qEOuYxGMPi4chUa1I9AMC7wM5slHUhm",
"12JCTv0B_YLcU_fnAjScakeKQLaG28YPJ", "167KwcwxIejRdpwVL43652SR6B7M49jb3", 
"1VRbQbMv4Jpa5eJ4H2rB_RVLMwqzmbKv2", "1gK3E5s3pSBlcjZ_ZKXk0c0f1C5rhZbaA",
"1S9XwLXdI5n1Qq0EN5Kji8UOSruFIzsy1", "1TsJLWgXukfqEz5IMJZ0baNU-clKeqV8g"]

#print(ce_jaccard_Ids)
print("Downloading Network Architecture And Weights...")

!gdown --id "1mtewaKMXqdIVmWj1mXKQfYK9Tr5OCD7J"
!gdown --id "1lQHqgcRCDhmAL5BrxSyfYwtejmpt_rXP"
!gdown --id "1K1cduESQHDozHZYInEE0PmREdvvahiRs"
!gdown --id "1rwJ9PKUuZAw9ZON9u6P7KC9H05jIT1nZ"
!gdown --id "1Tn0e3OPhmFjZXLksP2reSTls3CFwoXSv"
!gdown --id "1RrhIiRtWYFGJTpq-IRTXYyldaO0fKXFN"
!gdown --id "1-R1kK7Z89XVSSpraTS2DfT8nFXqek8Ar"
!gdown --id "127f8DRvLuJ33V5_i-TtOZ4-SKrPzsCsT"
!gdown --id "1EWPauaixDCOHcDdN8LyBYj_CrtgCn6MX"
!gdown --id "1N2e2sNJ8agWAiEX9Fw4gjEwk5OcFo_RY"

!gdown --id "1G4LfnQQzn0ufnJkWvpEGmbdqXCZ02YKn"
!gdown --id "1P16p6KZt04chEt8griUQc-A6n7f4H6O1"
!gdown --id "1znKMg6KqkNGniIRtL6d3qEV41ZAJWrG4"
!gdown --id "15qEOuYxGMPi4chUa1I9AMC7wM5slHUhm"
!gdown --id "12JCTv0B_YLcU_fnAjScakeKQLaG28YPJ"
!gdown --id "167KwcwxIejRdpwVL43652SR6B7M49jb3"
!gdown --id "1VRbQbMv4Jpa5eJ4H2rB_RVLMwqzmbKv2"
!gdown --id "1gK3E5s3pSBlcjZ_ZKXk0c0f1C5rhZbaA"
!gdown --id "1S9XwLXdI5n1Qq0EN5Kji8UOSruFIzsy1"
!gdown --id "1TsJLWgXukfqEz5IMJZ0baNU-clKeqV8g"

network_Names = os.listdir()
network_Names.remove('README.md')
network_Names.remove('.git')

phase_1_Network_Names = []
phase_2_Network_Names = []
for name in network_Names:
  if 'Ensemble' in name:
    phase_1_Network_Names.append(name)
  elif 'Ensemble' not in name:
    phase_2_Network_Names.append(name)

phase_1_Ensemble = []
phase_2_Ensemble = []

for name in phase_1_Network_Names:
  if '.json' in name:
    json_file = open(name)
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name[:-5] + ".h5")
    phase_1_Ensemble.append(loaded_model)

for name in phase_2_Network_Names:
  if '.json' in name:
    json_file = open(name)
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(name[:-5] + ".h5")
    phase_2_Ensemble.append(loaded_model)


print("Done!")

'''# Load Downloaded Network Architecture And Weights.
print("Loading Network Architecture and Weights...")
network_Name = "Motion_Blur_Network.h5"
network = tf.keras.models.load_model(network_Name)
print("Done!")'''

Importing Useful Libraries...
Done!
Cloning Repository...
Cloning into 'Deformable_Nondeformable'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.
Done!
/content/Deformable_Nondeformable
Downloading...
From: https://drive.google.com/uc?id=1mtewaKMXqdIVmWj1mXKQfYK9Tr5OCD7J
To: /content/Deformable_Nondeformable/Phase1-Ensemble_Kfold-model_0.h5
63.5MB [00:00, 88.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lQHqgcRCDhmAL5BrxSyfYwtejmpt_rXP
To: /content/Deformable_Nondeformable/Phase1-Ensemble_Kfold-model_0.json
100% 31.0k/31.0k [00:00<00:00, 24.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1K1cduESQHDozHZYInEE0PmREdvvahiRs
To: /content/Deformable_Nondeformable/Phase1-Ensemble_Kfold-model_1.h5
63.5MB [00:00, 94.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rwJ9PKUuZAw9ZON9u6P7KC9H05jIT1nZ
To: /content/Deformable_Nonde

'# Load Downloaded Network Architecture And Weights.\nprint("Loading Network Architecture and Weights...")\nnetwork_Name = "Motion_Blur_Network.h5"\nnetwork = tf.keras.models.load_model(network_Name)\nprint("Done!")'

## Create directory to hold channels for analysis.

This cell is used to create a directory that will hold channels that we want to analyze.

IMPORTANT: Once this cell is run, you will need to manually upload channel images to the directory titled "Motion_Blur/Channel_Directory". 

If you want to re-initialize the directory, then just re-run this code after analyzing your first batch of images.

In [ ]:
#@title <- Click here to run code. Double click this text to view code.
# Create A Directory To Hold Channels For Analysis.
channel_Directory = "Channel_Directory/"
if os.path.isdir(channel_Directory):
  shutil.rmtree(channel_Directory)
os.mkdir(channel_Directory)

## Channel Analysis.

This cell applies analysis to all of the channels uploaded to the "Motion_Blur/Channel_Directory" directory. After each channel is analyzed, the sRBC counts are appended to a data frame and displayed to the user.

After all channels are analyzed, the final dataframe is displayed to the user.

The results are also saved as a .csv file under the name "Analysis_Output.csv" in the Deformable_Nondeformable/ directory on Colab.

In [ ]:
#@title <- Click here to run code. Double click this text to view code.
# An array to hold pixel areas for all tested channels.
blob_Sizes = []
output_Data_Frame = pd.DataFrame(columns = ["File Name", "Number Of Adhered 'Cells'", "Deformable Counts", "Non-Deformable Counts", "Other", "Time (s)"])
# A function which applies a zero mean normalization to the input images.
def standard_norm(img):
    height, width, channels = img.shape
    for channel in range(channels):
        img[:,:,channel] = (img[:,:,channel] - np.mean(img[:,:,channel]))/np.std(img[:,:,channel])
    return img

# Thresholds to be tested.
thresholds = [90]
padding = 20
# This large for loop will analyze all of the channels uploaded to the Channel Directory.
for image_Name in os.listdir(channel_Directory):
    # If statement necessary to avoid possible errors during analysis on Google Colab.
    if ".ipynb" in image_Name:
      continue
    # Start time of analysis.
    start_Time = time.time()
    print("Analyzing " + image_Name[:-4])
    # Reading In Channel.
    full_Channel = plt.imread(channel_Directory + image_Name)
    # Convert grayscale image to RGB if the input channel is grayscale.
    if len(np.shape(full_Channel)) == 2:
        full_Channel = cv2.cvtColor(full_Channel, cv2.COLOR_GRAY2RGB)
    # Defining characteristics of the input channel.
    image_Height, image_Width, channels = np.shape(full_Channel)
    # The following if statements are used to resize the channel to have
    # dimensions which are evenly dividable by 150. To avoid as much distortion
    # as possible, we will resize either up or down in each dimensions, 
    # depending on which end the dimensions is closer to.
    if (image_Height % 150) < 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int(np.floor(image_Width/150))
    elif (image_Height % 150) >= 75 and (image_Width % 150) >= 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    elif (image_Height % 150) >= 75 and (image_Width % 150) < 75:
        full_Channel_Resized = cv2.resize(full_Channel,(int(np.floor(image_Width/150)*150), int((np.floor(image_Height/150) + 1)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int((np.floor(image_Height/150) + 1))
        horizontal_Tiles = int(np.floor(image_Width/150))
    else:
        full_Channel_Resized = cv2.resize(full_Channel,(int((np.floor(image_Width/150) + 1)*150), int(np.floor(image_Height/150)*150)), interpolation = cv2.INTER_CUBIC)
        vertical_Tiles = int(np.floor(image_Height/150))
        horizontal_Tiles = int((np.floor(image_Width/150) + 1))
    full_Channel_Resized_Borders = cv2.copyMakeBorder(full_Channel_Resized.copy(), padding, padding, padding, padding, cv2.BORDER_CONSTANT)
    # Defining characteristics of the resized input channel.
    image_Height_Resized, image_Width_Resized, channels = np.shape(full_Channel_Resized)
    # Creating an array which will hold predictions.
    output_Image = np.zeros((image_Height_Resized,image_Width_Resized))

    # The following chunk of code will make predictions, and create output
    # images and mask predictions.
    x_Slider = 0
    y_Slider = 0
    # Creating an array which will hold predictions for each tile.
    output_Array = np.zeros((128,128))
    # In the following for loops, we will slide through the input channel, 
    # tile by tile, and make predictions on each tile.
    for i in range(vertical_Tiles):
        x_Slider = 150*i
        # Sliding through all tiles in a row for each row.
        for j in range(horizontal_Tiles):
            y_Slider = 150*j
            # Resizing tile to required input size.
            current_Tile = full_Channel_Resized[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150,:]/255
            current_Tile = cv2.resize(current_Tile, (128,128), interpolation=cv2.INTER_AREA)

            # Normalizing the tile.
            current_Tile_Normalized = standard_norm(current_Tile.copy())
            current_Tile_Normalized = current_Tile_Normalized[None,:,:,:]
            sample, height, width, depth = current_Tile_Normalized.shape
            empty_mask = np.zeros((sample, height, width, depth))
            for kfold, model in enumerate(phase_1_Ensemble):
              if kfold == 0:
                output = model(current_Tile_Normalized)
              else:
                output += model(current_Tile_Normalized)
            output *= 1/len(phase_1_Ensemble)

            # Finding the prediction for each pixel in the tile.
            #output = np.argmax(output, axis = 3)
            #output = np.reshape(output, (128,128,1))[:,:,0]
            output_Max = np.argmax(output, axis = 3)
            output_Array = output_Max[0,:,:].astype('float32')
            #for i in range(128):
                #for j in range(128):
                    #output_Array[i,j] = np.argmax(output[0,i,j,:])
            
            # Resizing tile back to original size.
            output_Array = cv2.resize(output_Array,(150,150),interpolation = cv2.INTER_AREA)
            output_Image[x_Slider:x_Slider + 150, y_Slider: y_Slider + 150] = output_Array
            output_Array = np.zeros((128,128))
    # The following for loops binarize the output, after resizing distortion.
    output_Image = np.around(output_Image).astype(int)
    #for i in range(image_Height_Resized):
        #for j in range(image_Width_Resized):
            #output_Image[i,j] = int(round(output_Image[i,j],1))

    # Defining connected pixel regions in the final image.
    blobs, number_Of_Blobs = label(output_Image == 1) # whole channel image
    properties = measure.regionprops(blobs)
    centroids = [prop.centroid for prop in properties if prop.area > 50]
    #print(len(centroids))
    deformable = 0
    nondeformable = 0
    other = 0
    for i in range(len(centroids)):
        cell_Region = full_Channel_Resized_Borders[int(centroids[i][0]) - 16 + padding:int(centroids[i][0]) + 16 + padding,int(centroids[i][1]) - 16 + padding:int(centroids[i][1]) + 16 + padding,:]
        cell_Region_Resized = cv2.resize(cell_Region.copy().astype('float64'), (224,224), interpolation=cv2.INTER_LINEAR)/255
        #cell_Count = cell_Count + 1
        cell_Region_Resized_Normalized = standard_norm(cell_Region_Resized.copy())
        cell_Region_Resized_Normalized = cell_Region_Resized_Normalized[None,:,:,:]
        for kfold, model in enumerate(phase_2_Ensemble):
            if kfold == 0:
                percentages = model(cell_Region_Resized_Normalized)
            else:
                percentages += model(cell_Region_Resized_Normalized)
        percentages *= 1/len(phase_2_Ensemble)
        prediction = np.argmax(percentages)
        if percentages[0][0] > 0.4:
            deformable = deformable + 1
        if percentages[0][1] > 0.4:
            nondeformable = nondeformable + 1
        if percentages[0][2] > 0.9:
            other = other + 1
    #print("Deformable =  " + str(deformable))
    #print("Nondeformable = " + str(nondeformable))
    #print("Other = " + str(other))
    end_Time = time.time()
    time_Change = end_Time - start_Time
    #print(time_Change)
    print("======================================")
    output_Data_Frame = output_Data_Frame.append(pd.DataFrame([[image_Name,len(centroids),deformable,nondeformable,other,time_Change]], columns = ["File Name", "Number Of Adhered 'Cells'", "Deformable Counts", "Non-Deformable Counts", "Other", "Time (s)"]))
    display(output_Data_Frame)
print("Final Data")
display(output_Data_Frame)
output_Data_Frame.to_csv("Analysis_Output.csv")

Analyzing im2-reduced-darkgraybg


,File Name,Number Of Adhered 'Cells',Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,im2-reduced-darkgraybg.jpg,56,41,5,1,80.685389


Analyzing im3-reduced


,File Name,Number Of Adhered 'Cells',Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,im2-reduced-darkgraybg.jpg,56,41,5,1,80.685389
0,im3-reduced.jpg,62,61,0,1,87.855200


Analyzing im1


...

,File Name,Number Of Adhered 'Cells',Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,im2-reduced-darkgraybg.jpg,56,41,5,1,80.685389
0,im3-reduced.jpg,62,61,0,1,87.855200
0,im1.jpg,3,0,0,3,55.509236
0,im1-reduced.jpg,62,14,44,1,88.147368
0,im1-darkgraybg.jpg,3,0,0,3,55.316226
0,im3-darkgraybg.jpg,64,0,0,47,130.968845
0,im1-reduced-darkgraybg.jpg,62,10,40,2,90.505423
0,im2-darkgraybg.jpg,46,0,0,34,109.546880
0,im3.jpg,60,1,2,36,127.739716
0,im3-reduced-darkgraybg.jpg,62,57,0,1,88.853361


Analyzing im2


,File Name,Number Of Adhered 'Cells',Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,im2-reduced-darkgraybg.jpg,56,41,5,1,80.685389
0,im3-reduced.jpg,62,61,0,1,87.855200
0,im1.jpg,3,0,0,3,55.509236
0,im1-reduced.jpg,62,14,44,1,88.147368
0,im1-darkgraybg.jpg,3,0,0,3,55.316226
0,im3-darkgraybg.jpg,64,0,0,47,130.968845
0,im1-reduced-darkgraybg.jpg,62,10,40,2,90.505423
0,im2-darkgraybg.jpg,46,0,0,34,109.546880
0,im3.jpg,60,1,2,36,127.739716
0,im3-reduced-darkgraybg.jpg,62,57,0,1,88.853361


Final Data


,File Name,Number Of Adhered 'Cells',Deformable Counts,Non-Deformable Counts,Other,Time (s)
0,im2-reduced-darkgraybg.jpg,56,41,5,1,80.685389
0,im3-reduced.jpg,62,61,0,1,87.855200
0,im1.jpg,3,0,0,3,55.509236
0,im1-reduced.jpg,62,14,44,1,88.147368
0,im1-darkgraybg.jpg,3,0,0,3,55.316226
0,im3-darkgraybg.jpg,64,0,0,47,130.968845
0,im1-reduced-darkgraybg.jpg,62,10,40,2,90.505423
0,im2-darkgraybg.jpg,46,0,0,34,109.546880
0,im3.jpg,60,1,2,36,127.739716
0,im3-reduced-darkgraybg.jpg,62,57,0,1,88.853361
